# Procesamiento de Lenguaje Natural
Tecnicatura Universitaria en Inteligencia Artificial

Materia: IA4.2

**Trabajo Práctico 2**


**Docentes:**
- D'Alessandro, Ariel
- Geary, Alan
- Leon Cavallo, Andrea
- Manson, Juan Pablo


**Integrantes:**

| Apellido y Nombre | Legajo   |
| ----------------- | -------- |
| Aguirre, Fabián   | A-4516/1 |


Año: 2023

## Enunciado

# Pautas generales:

•	El trabajo deberá ser realizado individualmente.

•	Deberá informar cuál es la url del repositorio con el que van a trabajar y las definiciones de en qué problematicas quisieran solucionar con un sistema multiagente en el siguiente formulario:  https://docs.google.com/forms/d/e/1FAIpQLSdOZNGOzQ1gbf43caA4ygAbLx5tm5bU-s8RdfdftOzd_aXzhA/viewform?usp=pp_url

•	Se debe entregar un informe en el cual se incluya las justificaciones y un vínculo a los archivos que permitan reproducir el proyecto.

•	Temas deseables a cubrir en el tp

  * Recuperación de datos de bases de datos de grafos

  * Extracción de conocimiento de texto y posterior inserción en una base de datos de grafos

  * Agentes (estará cubierto en el Ejercicio 2)

# Ejercicio 1 - RAG

Crear un chatbot experto en un tema a elección, usando la técnica RAG (Retrieval Augmented Generation). Como fuentes de conocimiento se utilizarán al menos las siguientes fuentes:

•	Documentos de texto

•	Datos numéricos en formato tabular (por ej., Dataframes, CSV, sqlite, etc.)

•	Base de datos de grafos (Online o local)

El sistema debe poder llevar a cabo una conversación en lenguaje español. El usuario podrá hacer preguntas, que el chatbot intentará responder a partir de datos de algunas de sus fuentes. El asistente debe poder clasificar las preguntas, para saber qué fuentes de datos utilizar como contexto para generar una respuesta.

## Requerimientos generales

•	Realizar todo el proyecto en un entorno Google Colab

•	El conjunto de datos debe tener al menos 100 páginas de texto y un mínimo de 3 documentos.

•	Realizar split de textos usando Langchain (RecursiveTextSearch, u otros
métodos disponibles). Limpiar el texto según sea conveniente.

•	Realizar los embeddings que permitan vectorizar el texto y almacenarlo en una base de datos ChromaDB

•	Los modelos de embeddings y LLM para generación de texto son a elección


# Ejercicio 2 - Agentes

Realice una investigación respecto al estado del arte de las aplicaciones actuales de agentes inteligentes usando modelos LLM libres.

Plantee una problemática a solucionar con un sistema multiagente. Defina cada uno de los agentes involucrados en la tarea.

Realice un informe con los resultados de la investigación y con el esquema del sistema multiagente, no olvide incluir fuentes de información.

Opcional: Resolución con código de dicho escenario.



# Resolución

In [ ]:
!pip install gdown
!pip install pymupdf
!pip install rdflib
!pip install pypdf langchain python-decouple
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install chromadb

#!pip install sentence-transformers

  Using cached PyMuPDF-1.23.7-cp310-none-manylinux2014_x86_64.whl (4.4 MB)
  Using cached PyMuPDFb-1.23.7-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (30.6 MB)


In [ ]:
import gdown
import os
import shutil
import sys
import pandas as pd
import tensorflow_text
import tensorflow_hub as hub
import langchain
import chromadb
import fitz
import re
import requests

from jinja2 import Template
from decouple import config
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rdflib import Graph, URIRef, Literal, Namespace

Descarga de documentos

In [ ]:
url = 'https://drive.google.com/drive/folders/1xLQnsyMX8vCWplS3RXqkVcdRdn42GhSA?usp=sharing'

gdown.download_folder(url, quiet=True, output='tp2-nlp')

carpeta_destino = 'documentos'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

carpeta_origen = 'tp2-nlp/tp2-nlp-documentos'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

shutil.rmtree(carpeta_origen)

carpeta_origen = 'tp2-nlp'
carpeta_destino = '/content/'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

shutil.rmtree(carpeta_origen)

sys.path.append('/content/')

print("Archivos descargados con éxito.")

Archivos descargados con éxito.


In [ ]:
#Creación de la base con CrhomaDB
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

client = chromadb.Client()
collection = client.get_or_create_collection("all-my-documents")

pdf_directory = "documentos"

pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]

textos = []
ids_textos = []
fuentes = []

def clean_text(text):
    cleaned_text = text.lower()
    cleaned_text = re.sub(r'[^\w\s.,]', '', cleaned_text)
    return cleaned_text

splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=10)

for i, pdf_file in enumerate(pdf_files, start=1):
    pdf_path = os.path.join(pdf_directory, pdf_file)

    with fitz.open(pdf_path) as doc:
        text = ""
        for page_num in range(doc.page_count):
            page = doc[page_num]
            text += page.get_text()

    split_texts = splitter.split_text(text)

    clean_texts = [clean_text(sentence) for sentence in split_texts]

    textos.extend(clean_texts)
    ids_textos.extend([f"doc{i}_{j}" for j in range(1, len(clean_texts) + 1)])
    fuentes.extend([f"fuente{i}" for _ in range(len(clean_texts))])

embeddings = embed(textos).numpy().tolist()

collection.add(
    documents=textos,
    metadatas=[{"source": fuente} for fuente in fuentes],
    ids=ids_textos,
    embeddings=embeddings)
#client.delete_collection('all-my-documents')

In [ ]:
#Creación del grafo de tasa_vih
g = Graph()

n = Namespace("http://example.org/place/")
EX = Namespace("http://example.org/terms/")

csv_path = "tasa_vih_jurid.csv"
df = pd.read_csv(csv_path, encoding='utf-8-sig')

def create_rdf_from_data(graph, row, n, EX):
    subject = row["jurisdiccion"]
    obj = row["tasa_vih"]
    relation = "tiene_tasa_vih"

    subject_uri = URIRef(n + f"entity_{subject}")

    graph.add((subject_uri, EX.jurisdiccion, Literal(subject)))
    graph.add((subject_uri, EX.tasa_vih, Literal(obj)))

for _, row in df.iterrows():
    create_rdf_from_data(g, row, n, EX)

g.serialize("tasa-vih-jurid.rdf", format="turtle")

<Graph identifier=N28ace174ab884f77807a224d6ae7680a (<class 'rdflib.graph.Graph'>)>

In [ ]:
#CHATBOT

# Cargar el RDF creado anteriormente
graph = Graph()
graph.parse("tasa-vih-jurid.rdf", format="turtle")

def query_chromadb(query_str: str):
    results = collection.query(
        query_embeddings=embed([query_str]).numpy().tolist(),
        n_results=20
    )
    return results


def zephyr_instruct_template(messages):

    template_str = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"

    template = Template(template_str)

    return template.render(messages=messages)


def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    try:

        api_key = config('HUGGINGFACE_TOKEN')

        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        headers = {"Authorization": f"Bearer {api_key}"}

        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        response = requests.post(api_url, headers=headers, json=data)

        respuesta = response.json()[0]["generated_text"][len(prompt):]

        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")


def prepare_prompt(query_str: str):

    TEXT_QA_PROMPT_TMPL = (
        "La información de contexto es la siguiente:\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n"
        "Pregunta: {query_str}\n"
        "Respuesta: "
    )
    # Se agrega al contexto el contenido extraido de CrhomaDB
    documents = query_chromadb(query_str)

    context_str = ''

    for documents_lista in documents["documents"]:
        for doc in documents_lista:
            context_str += f"{doc}\n"

    # Se agrega al contexto el contenido del grafo
    q = """
        SELECT ?juris ?tasa_vih WHERE {
            ?juris <http://example.org/terms/tasa_vih> ?tasa_vih .
        }
        """

    results = graph.query(q)

    for r in results:
        juris = r['juris'].split("entity_")[-1]
        context_str += f"{juris} tiene una tasa de VIH de {r['tasa_vih']}\n"

    messages = [
        {
            "role": "system",
            "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
        },
        {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)},
    ]

    final_prompt = zephyr_instruct_template(messages)
    return final_prompt

print('**BIENVENIDOS AL CHATBOT DE SALUD Y ALIMENTACIÓN**\n')
print('Las preguntas puede estar relacionadas con los siguiente temas: \n')
print('-Alimentación basada en plantas\n')
print('-Entornos escolares saludables\n')
print('-Ética y transplantes\n')
print('-Sodio en la alimentación\n')
print('-Tasa de VIH por provincias\n')

while True:
    user_query = input("Ingrese su consulta (o 'salir' para salir): ")
    if user_query.lower() == 'salir':
        break

    final_prompt = prepare_prompt(user_query)
    print('Respuesta:')
    print(generate_answer(final_prompt))
    print('-------------------------------------------------------')

#Ejemplos de preguntas
#¿Cuál es la tasa de VIH de Neuquen?
#¿Cómo es el proceso de transplante de organos?
#¿Qué puede producir en el cuerpo el exceso de consumo de sodio en los alimentos?
#¿Cuáles son los requerimientos para personas con alimentación vegetariana?

**BIENVENIDOS AL CHATBOT DE SALUD Y ALIMENTACIÓN**

Las preguntas puede estar relacionadas con los siguiente temas: 

-Alimentación basada en plantas

-Entornos escolares saludables

-Ética y transplantes

-Sodio en la alimentación

-Tasa de VIH por provincias

Respuesta:
<|assistant|>
La tasa de VIH en Neuquen es de 12.2, según la información de contexto proporcionada.
-------------------------------------------------------
Respuesta:
<|assistant|>
El proceso de transplante de órganos se lleva a cabo en personas que necesitan reemplazar uno o más de sus órganos debido a enfermedades o lesiones graves. El donante debe ser un individuo vivo o un cadáver recientemente fallecido, y los órganos y tejidos se seleccionan y evaluan para determinar si son aptos para el trasplante. El proceso de transplante implica la cirugía para extraer el órgano del donante y luego implantarlo en el receptor. Los órganos más comunes para el trasplante incluyen el riñón, el corazón, el hígado, y los pulmones,